<a href="https://colab.research.google.com/github/teamgaon/SANUP/blob/main/220406_hj_SANUP_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리, 패키지

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers

     |████████████████████████████████| 325 kB 5.1 MB/s 
     |████████████████████████████████| 3.8 MB 73.1 MB/s 
     |████████████████████████████████| 136 kB 77.8 MB/s 
     |████████████████████████████████| 212 kB 78.1 MB/s 
     |████████████████████████████████| 1.1 MB 64.7 MB/s 
     |████████████████████████████████| 77 kB 5.9 MB/s 
     |████████████████████████████████| 127 kB 80.6 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 271 kB 80.9 MB/s 
     |████████████████████████████████| 144 kB 61.8 MB/s 
     |████████████████████████████████| 596 kB 67.9 MB/s 
     |████████████████████████████████| 6.5 MB 71.6 MB/s 
     |████████████████████████████████| 895 kB 71.3 MB/s 
     |████████████████████████████████| 1.2 MB 68.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
   

- TPU 사용을 위한 accelerate

In [3]:
from accelerate import Accelerator

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from transformers import TrainingArguments, Trainer
from transformers import AutoModel,AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import gc
from transformers import AdamW
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup
from tqdm.auto import tqdm
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')
from accelerate import notebook_launcher
from pathlib import Path
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv('/content/drive/MyDrive/SANUP/1. 실습용자료.txt', sep='|', encoding='cp949')
test = pd.read_csv('/content/drive/MyDrive/SANUP/2. 모델개발용자료.txt', sep='|', encoding='cp949')
submission = pd.read_csv('/content/drive/MyDrive/SANUP/답안 작성용 파일.csv', encoding='cp949')

In [6]:
df = df.fillna('')

In [7]:
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육
...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스


In [8]:
df = df.sample(n=100000)
df = df.reset_index(drop=True)

In [9]:
df_train = df[:90000]
df_test_1 = df[90000:]
df_test = df_test_1

In [10]:
df_test['text'] = df_test['text_obj'].astype(str) + " " + df_test['text_mthd'].astype(str) + " " + df_test['text_deal'].astype(str)
df_test = df_test[['text']]
df_test = df_test.reset_index(drop=True)

In [11]:
df_test

,text
0,소매업자를 대상으로 사업장에서 반려동물 사료 등을 판매
1,접객시설을 갖추고 김밥등 분식 제공
2,개인택시로 일반인을 대상으로 승객운송서비스
3,현장에서 고객의 의뢰에 의해 기와로 지붕공사
4,가정용 철물및 용품을 수리 시설을 갖추고 가정용 보일러 및 하수도를 수리
...,...
9995,펜션에서 일반인을 대상으로 숙박서비스제공
9996,음식점에서 접객시설을 갖추고 추어탕 판매
9997,부동산중개소 계약에의거 부동산중개 부동산중개 서비스
9998,"원단,실 재단,재봉하여 남성복"


### digit_1 예측하기

In [12]:
df_digit1 = df_train

In [13]:
df_digit1['text'] = df_digit1['text_obj'].astype(str) + " " + df_digit1['text_mthd'].astype(str) + " " + df_digit1['text_deal'].astype(str)
df_digit1['target'] = df_digit1['digit_1']
df_digit1

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,target
0,id_0751277,G,47,471,일반소비자에게,"슈퍼마켓(165-3,000㎡미만)에서",각종상품소매,"일반소비자에게 슈퍼마켓(165-3,000㎡미만)에서 각종상품소매",G
1,id_0034836,G,47,473,소매업,일반소비자에게,휴대폰,소매업 일반소비자에게 휴대폰,G
2,id_0137579,S,96,961,미용실에서,고객을 대상으로,"커트, 파마, 염색","미용실에서 고객을 대상으로 커트, 파마, 염색",S
3,id_0917187,G,46,463,대리점,소매업자에게 도매,각종음료수,대리점 소매업자에게 도매 각종음료수,G
4,id_0434504,J,58,581,학생을 대상으로 교과서 및 학습서적 출판업,,,학생을 대상으로 교과서 및 학습서적 출판업,J
...,...,...,...,...,...,...,...,...,...
89995,id_0284399,G,46,465,사업장에서,병원대상으로,의료기납품,사업장에서 병원대상으로 의료기납품,G
89996,id_0784081,H,49,493,용달화물자동차로,고객의 요청으로,화물운송서비스,용달화물자동차로 고객의 요청으로 화물운송서비스,H
89997,id_0251254,C,28,283,전선,"절단, 포장",전기용품,"전선 절단, 포장 전기용품",C
89998,id_0144449,F,42,424,얄미늄 샷시문,고객의 요청으로,금속창호설치공사,얄미늄 샷시문 고객의 요청으로 금속창호설치공사,F


In [14]:
train_digit1 = df_digit1[['text', 'target']]
train_digit1

,text,target
0,"일반소비자에게 슈퍼마켓(165-3,000㎡미만)에서 각종상품소매",G
1,소매업 일반소비자에게 휴대폰,G
2,"미용실에서 고객을 대상으로 커트, 파마, 염색",S
3,대리점 소매업자에게 도매 각종음료수,G
4,학생을 대상으로 교과서 및 학습서적 출판업,J
...,...,...
89995,사업장에서 병원대상으로 의료기납품,G
89996,용달화물자동차로 고객의 요청으로 화물운송서비스,H
89997,"전선 절단, 포장 전기용품",C
89998,얄미늄 샷시문 고객의 요청으로 금속창호설치공사,F


### 1. digit_2 예측하기

In [15]:
df_digit2 = df_train

In [16]:
df_digit2['text'] = df_digit2['text_obj'].astype(str) + " " + df_digit2['text_mthd'].astype(str) + " " + df_digit2['text_deal'].astype(str)
df_digit2['target'] = df_digit2['digit_2'].astype(str)
df_digit2

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,target
0,id_0751277,G,47,471,일반소비자에게,"슈퍼마켓(165-3,000㎡미만)에서",각종상품소매,"일반소비자에게 슈퍼마켓(165-3,000㎡미만)에서 각종상품소매",47
1,id_0034836,G,47,473,소매업,일반소비자에게,휴대폰,소매업 일반소비자에게 휴대폰,47
2,id_0137579,S,96,961,미용실에서,고객을 대상으로,"커트, 파마, 염색","미용실에서 고객을 대상으로 커트, 파마, 염색",96
3,id_0917187,G,46,463,대리점,소매업자에게 도매,각종음료수,대리점 소매업자에게 도매 각종음료수,46
4,id_0434504,J,58,581,학생을 대상으로 교과서 및 학습서적 출판업,,,학생을 대상으로 교과서 및 학습서적 출판업,58
...,...,...,...,...,...,...,...,...,...
89995,id_0284399,G,46,465,사업장에서,병원대상으로,의료기납품,사업장에서 병원대상으로 의료기납품,46
89996,id_0784081,H,49,493,용달화물자동차로,고객의 요청으로,화물운송서비스,용달화물자동차로 고객의 요청으로 화물운송서비스,49
89997,id_0251254,C,28,283,전선,"절단, 포장",전기용품,"전선 절단, 포장 전기용품",28
89998,id_0144449,F,42,424,얄미늄 샷시문,고객의 요청으로,금속창호설치공사,얄미늄 샷시문 고객의 요청으로 금속창호설치공사,42


In [17]:
train_digit2 = df_digit2[['text', 'target']]
train_digit2

,text,target
0,"일반소비자에게 슈퍼마켓(165-3,000㎡미만)에서 각종상품소매",47
1,소매업 일반소비자에게 휴대폰,47
2,"미용실에서 고객을 대상으로 커트, 파마, 염색",96
3,대리점 소매업자에게 도매 각종음료수,46
4,학생을 대상으로 교과서 및 학습서적 출판업,58
...,...,...
89995,사업장에서 병원대상으로 의료기납품,46
89996,용달화물자동차로 고객의 요청으로 화물운송서비스,49
89997,"전선 절단, 포장 전기용품",28
89998,얄미늄 샷시문 고객의 요청으로 금속창호설치공사,42


### digit_3 예측하기

In [18]:
df_digit3 = df_train

In [19]:
df_digit3['text'] = df_digit3['text_obj'].astype(str) + " " + df_digit3['text_mthd'].astype(str) + " " + df_digit3['text_deal'].astype(str)
df_digit3['target'] = df_digit3['digit_3'].astype(str)
df_digit3

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,target
0,id_0751277,G,47,471,일반소비자에게,"슈퍼마켓(165-3,000㎡미만)에서",각종상품소매,"일반소비자에게 슈퍼마켓(165-3,000㎡미만)에서 각종상품소매",471
1,id_0034836,G,47,473,소매업,일반소비자에게,휴대폰,소매업 일반소비자에게 휴대폰,473
2,id_0137579,S,96,961,미용실에서,고객을 대상으로,"커트, 파마, 염색","미용실에서 고객을 대상으로 커트, 파마, 염색",961
3,id_0917187,G,46,463,대리점,소매업자에게 도매,각종음료수,대리점 소매업자에게 도매 각종음료수,463
4,id_0434504,J,58,581,학생을 대상으로 교과서 및 학습서적 출판업,,,학생을 대상으로 교과서 및 학습서적 출판업,581
...,...,...,...,...,...,...,...,...,...
89995,id_0284399,G,46,465,사업장에서,병원대상으로,의료기납품,사업장에서 병원대상으로 의료기납품,465
89996,id_0784081,H,49,493,용달화물자동차로,고객의 요청으로,화물운송서비스,용달화물자동차로 고객의 요청으로 화물운송서비스,493
89997,id_0251254,C,28,283,전선,"절단, 포장",전기용품,"전선 절단, 포장 전기용품",283
89998,id_0144449,F,42,424,얄미늄 샷시문,고객의 요청으로,금속창호설치공사,얄미늄 샷시문 고객의 요청으로 금속창호설치공사,424


In [20]:
train_digit3 = df_digit3[['text', 'target']]
train_digit3

,text,target
0,"일반소비자에게 슈퍼마켓(165-3,000㎡미만)에서 각종상품소매",471
1,소매업 일반소비자에게 휴대폰,473
2,"미용실에서 고객을 대상으로 커트, 파마, 염색",961
3,대리점 소매업자에게 도매 각종음료수,463
4,학생을 대상으로 교과서 및 학습서적 출판업,581
...,...,...
89995,사업장에서 병원대상으로 의료기납품,465
89996,용달화물자동차로 고객의 요청으로 화물운송서비스,493
89997,"전선 절단, 포장 전기용품",283
89998,얄미늄 샷시문 고객의 요청으로 금속창호설치공사,424


# 토크나이저, 함수

In [21]:
from transformers import AutoTokenizer
from transformers import ElectraModel, ElectraTokenizer

checkpoint = "monologg/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

In [22]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

### digit_1

In [23]:
my_dict1 = {}
for i in range(len(train_digit1['target'].unique())):
  my_dict1[train_digit1['target'].unique()[i]] = i

def target_to_num_digit1(target:str):
  return my_dict1[target]

train_digit1['target'] = train_digit1['target'].map(target_to_num_digit1)
train_digit1

,text,target
0,"일반소비자에게 슈퍼마켓(165-3,000㎡미만)에서 각종상품소매",0
1,소매업 일반소비자에게 휴대폰,0
2,"미용실에서 고객을 대상으로 커트, 파마, 염색",1
3,대리점 소매업자에게 도매 각종음료수,0
4,학생을 대상으로 교과서 및 학습서적 출판업,2
...,...,...
89995,사업장에서 병원대상으로 의료기납품,0
89996,용달화물자동차로 고객의 요청으로 화물운송서비스,8
89997,"전선 절단, 포장 전기용품",9
89998,얄미늄 샷시문 고객의 요청으로 금속창호설치공사,7


In [24]:
my_dict1[''] = len(my_dict1)

In [25]:
train_digit1[train_digit1['text'].isnull()]

,text,target


### digit_2

In [26]:
my_dict2 = {}
for i in range(len(train_digit2['target'].unique())):
  my_dict2[train_digit2['target'].unique()[i]] = i

def target_to_num_digit2(target:str):
  return my_dict2[target]

train_digit2['target'] = train_digit2['target'].map(target_to_num_digit2)
train_digit2

,text,target
0,"일반소비자에게 슈퍼마켓(165-3,000㎡미만)에서 각종상품소매",0
1,소매업 일반소비자에게 휴대폰,0
2,"미용실에서 고객을 대상으로 커트, 파마, 염색",1
3,대리점 소매업자에게 도매 각종음료수,2
4,학생을 대상으로 교과서 및 학습서적 출판업,3
...,...,...
89995,사업장에서 병원대상으로 의료기납품,2
89996,용달화물자동차로 고객의 요청으로 화물운송서비스,9
89997,"전선 절단, 포장 전기용품",29
89998,얄미늄 샷시문 고객의 요청으로 금속창호설치공사,8


In [27]:
my_dict2[''] = len(my_dict2)

In [28]:
train_digit2[train_digit2['text'].isnull()]

,text,target


### digit_3

In [29]:
my_dict3 = {}
for i in range(len(train_digit3['target'].unique())):
  my_dict3[train_digit3['target'].unique()[i]] = i

def target_to_num_digit3(target:str):
  return my_dict3[target]

train_digit3['target'] = train_digit3['target'].map(target_to_num_digit3)
train_digit3

,text,target
0,"일반소비자에게 슈퍼마켓(165-3,000㎡미만)에서 각종상품소매",0
1,소매업 일반소비자에게 휴대폰,1
2,"미용실에서 고객을 대상으로 커트, 파마, 염색",2
3,대리점 소매업자에게 도매 각종음료수,3
4,학생을 대상으로 교과서 및 학습서적 출판업,4
...,...,...
89995,사업장에서 병원대상으로 의료기납품,13
89996,용달화물자동차로 고객의 요청으로 화물운송서비스,25
89997,"전선 절단, 포장 전기용품",167
89998,얄미늄 샷시문 고객의 요청으로 금속창호설치공사,23


In [30]:
my_dict3[''] = len(my_dict3)

In [31]:
train_digit3[train_digit3['text'].isnull()]

,text,target


In [32]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [33]:
display(train_digit1)
display(train_digit2)
display(train_digit3)

,text,target
0,"일반소비자에게 슈퍼마켓(165-3,000㎡미만)에서 각종상품소매",0
1,소매업 일반소비자에게 휴대폰,0
2,"미용실에서 고객을 대상으로 커트, 파마, 염색",1
3,대리점 소매업자에게 도매 각종음료수,0
4,학생을 대상으로 교과서 및 학습서적 출판업,2
...,...,...
89995,사업장에서 병원대상으로 의료기납품,0
89996,용달화물자동차로 고객의 요청으로 화물운송서비스,8
89997,"전선 절단, 포장 전기용품",9
89998,얄미늄 샷시문 고객의 요청으로 금속창호설치공사,7


,text,target
0,"일반소비자에게 슈퍼마켓(165-3,000㎡미만)에서 각종상품소매",0
1,소매업 일반소비자에게 휴대폰,0
2,"미용실에서 고객을 대상으로 커트, 파마, 염색",1
3,대리점 소매업자에게 도매 각종음료수,2
4,학생을 대상으로 교과서 및 학습서적 출판업,3
...,...,...
89995,사업장에서 병원대상으로 의료기납품,2
89996,용달화물자동차로 고객의 요청으로 화물운송서비스,9
89997,"전선 절단, 포장 전기용품",29
89998,얄미늄 샷시문 고객의 요청으로 금속창호설치공사,8


,text,target
0,"일반소비자에게 슈퍼마켓(165-3,000㎡미만)에서 각종상품소매",0
1,소매업 일반소비자에게 휴대폰,1
2,"미용실에서 고객을 대상으로 커트, 파마, 염색",2
3,대리점 소매업자에게 도매 각종음료수,3
4,학생을 대상으로 교과서 및 학습서적 출판업,4
...,...,...
89995,사업장에서 병원대상으로 의료기납품,13
89996,용달화물자동차로 고객의 요청으로 화물운송서비스,25
89997,"전선 절단, 포장 전기용품",167
89998,얄미늄 샷시문 고객의 요청으로 금속창호설치공사,23


# 학습

* Model
  + Roberta-large

* Optimizer
  + AdamW

* Learning rate scheduler
  + Cosine annealing with warmup

### digit_1

In [35]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=32, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=32, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = len(train_digit1['target'].unique())
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 10
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )
  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/220406_inference_digit1/model' + str(fold), save_function=accelerator.save)

In [36]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print('--------------------------------')

tokenized_train = tokenizer(
  list(train_digit1['text']),
  return_tensors="pt",
  max_length=64, # Max_Length = 64
  padding=True,
  truncation=True,
  add_special_tokens=True
  )
  
for fold, (train_ids, test_ids) in enumerate(kfold.split(train_digit1, train_digit1['target'])):
  my_file = Path("/content/drive/MyDrive/220406_inference_digit1/model"+str(fold))
  if my_file.is_dir():
    continue

  print(f'FOLD {fold}')

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

  train_label = train_digit1['target']

  train_dataset = BERTDataset(tokenized_train, train_label)

  notebook_launcher(training_function)

--------------------------------
FOLD 0
Launching a training on 8 TPU cores.


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

...

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

epoch 1 train acc 0.6210106382978723
epoch 1 train acc 0.615802304964539
epoch 1 train acc 0.6257757092198581
epoch 1 train acc 0.6211214539007093
epoch 1 train acc 0.6315381205673759
epoch 1 train acc 0.6275487588652482
epoch 1 train acc 0.6216755319148937
epoch 1 train acc 0.6234485815602837
epoch 1 test acc 0.8882042253521126
epoch 1 test acc 0.8930457746478874
epoch 1 test acc 0.8890845070422535
epoch 1 test acc 0.8904049295774648
epoch 1 test acc 0.8785211267605634
epoch 1 test acc 0.8926056338028169
epoch 1 test acc 0.8824823943661971
epoch 1 test acc 0.883362676056338
epoch 2 train acc 0.9113475177304965
epoch 2 train acc 0.9144503546099291
epoch 2 train acc 0.910904255319149
epoch 2 train acc 0.9115691489361702
epoch 2 train acc 0.9076906028368794
epoch 2 train acc 0.9123448581560284
epoch 2 train acc 0.9137854609929078
epoch 2 train acc 0.9056959219858156
epoch 2 test acc 0.9352992957746479
epoch 2 test acc 0.9238556338028169
epoch 2 test acc 0.9339788732394366
epoch 2 test ac

In [37]:
train_digit1

,text,target
0,"일반소비자에게 슈퍼마켓(165-3,000㎡미만)에서 각종상품소매",0
1,소매업 일반소비자에게 휴대폰,0
2,"미용실에서 고객을 대상으로 커트, 파마, 염색",1
3,대리점 소매업자에게 도매 각종음료수,0
4,학생을 대상으로 교과서 및 학습서적 출판업,2
...,...,...
89995,사업장에서 병원대상으로 의료기납품,0
89996,용달화물자동차로 고객의 요청으로 화물운송서비스,8
89997,"전선 절단, 포장 전기용품",9
89998,얄미늄 샷시문 고객의 요청으로 금속창호설치공사,7


### digit_2

In [38]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=32, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=32, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = len(train_digit2['target'].unique())
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 10
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )
  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/220406_inference_digit2/model' + str(fold), save_function=accelerator.save)

In [39]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print('--------------------------------')

tokenized_train = tokenizer(
  list(train_digit2['text']),
  return_tensors="pt",
  max_length=64, # Max_Length = 64
  padding=True,
  truncation=True,
  add_special_tokens=True
  )
  
for fold, (train_ids, test_ids) in enumerate(kfold.split(train_digit2, train_digit2['target'])):
  my_file = Path("/content/drive/MyDrive/220406_inference_digit2/model"+str(fold))
  if my_file.is_dir():
    continue

  print(f'FOLD {fold}')

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

  train_label2 = train_digit2['target']

  train_dataset2 = BERTDataset(tokenized_train, train_label2)

  notebook_launcher(training_function)

--------------------------------
FOLD 0
Launching a training on 8 TPU cores.


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

...

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

epoch 1 train acc 0.5241578014184397
epoch 1 train acc 0.5260416666666666
epoch 1 train acc 0.5179521276595744
epoch 1 train acc 0.5155141843971631
epoch 1 train acc 0.5254875886524822
epoch 1 train acc 0.5255984042553191
epoch 1 train acc 0.5198359929078015
epoch 1 train acc 0.520279255319149
epoch 1 test acc 0.8036971830985915
epoch 1 test acc 0.815580985915493
epoch 1 test acc 0.797975352112676
epoch 1 test acc 0.8199823943661971
epoch 1 test acc 0.8142605633802817
epoch 1 test acc 0.8186619718309859
epoch 1 test acc 0.8058978873239436
epoch 1 test acc 0.7953345070422535
epoch 2 train acc 0.8725620567375887
epoch 2 train acc 0.879654255319149
epoch 2 train acc 0.882313829787234
epoch 2 train acc 0.8763297872340425
epoch 2 train acc 0.8828679078014184
epoch 2 train acc 0.8822030141843972
epoch 2 train acc 0.8830895390070922
epoch 2 train acc 0.8720079787234043
epoch 2 test acc 0.9264964788732394
epoch 2 test acc 0.9115316901408451
epoch 2 test acc 0.9185739436619719
epoch 2 test acc 

### digit_3

In [40]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=32, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=32, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = len(train_digit3['target'].unique())
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 10
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )
  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/220406_inference_digit3/model' + str(fold), save_function=accelerator.save)

In [41]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print('--------------------------------')

tokenized_train = tokenizer(
  list(train_digit3['text']),
  return_tensors="pt",
  max_length=64, # Max_Length = 64
  padding=True,
  truncation=True,
  add_special_tokens=True
  )
  
for fold, (train_ids, test_ids) in enumerate(kfold.split(train_digit3, train_digit3['target'])):
  my_file = Path("/content/drive/MyDrive/220406_inference_digit3/model"+str(fold))
  if my_file.is_dir():
    continue

  print(f'FOLD {fold}')

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

  train_label3 = train_digit3['target']

  train_dataset3 = BERTDataset(tokenized_train, train_label3)

  notebook_launcher(training_function)

--------------------------------
FOLD 0
Launching a training on 8 TPU cores.


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

  0%|          | 0/2820 [00:00<?, ?it/s]

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

  0%|          | 0/2820 [00:00<?, ?it/s]

...

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

epoch 1 train acc 0.49756205673758863
epoch 1 train acc 0.5009973404255319
epoch 1 train acc 0.49157801418439717
epoch 1 train acc 0.4947916666666667
epoch 1 train acc 0.4853723404255319
epoch 1 train acc 0.4859264184397163
epoch 1 train acc 0.490802304964539
epoch 1 train acc 0.49645390070921985
epoch 1 test acc 0.7161091549295775
epoch 1 test acc 0.7253521126760564
epoch 1 test acc 0.7090669014084507
epoch 1 test acc 0.7068661971830986
epoch 1 test acc 0.7262323943661971
epoch 1 test acc 0.7191901408450704
epoch 1 test acc 0.7147887323943662
epoch 1 test acc 0.7165492957746479
epoch 2 train acc 0.8201462765957447
epoch 2 train acc 0.8087322695035462
epoch 2 train acc 0.8179299645390071
epoch 2 train acc 0.8077349290780141
epoch 2 train acc 0.8136081560283688
epoch 2 train acc 0.8188164893617021
epoch 2 train acc 0.8150487588652482
epoch 2 train acc 0.821697695035461
epoch 2 test acc 0.9080105633802817
epoch 2 test acc 0.9040492957746479
epoch 2 test acc 0.9119718309859155
epoch 2 tes

In [42]:
train_digit3['target'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

# 추론

### digit_1

In [43]:
df_test_digit1 = df_test
df_test_digit1

,text
0,소매업자를 대상으로 사업장에서 반려동물 사료 등을 판매
1,접객시설을 갖추고 김밥등 분식 제공
2,개인택시로 일반인을 대상으로 승객운송서비스
3,현장에서 고객의 의뢰에 의해 기와로 지붕공사
4,가정용 철물및 용품을 수리 시설을 갖추고 가정용 보일러 및 하수도를 수리
...,...
9995,펜션에서 일반인을 대상으로 숙박서비스제공
9996,음식점에서 접객시설을 갖추고 추어탕 판매
9997,부동산중개소 계약에의거 부동산중개 부동산중개 서비스
9998,"원단,실 재단,재봉하여 남성복"


In [44]:
df_test_digit1['target'] = ''

In [46]:
df_test_digit1

,text,target
0,소매업자를 대상으로 사업장에서 반려동물 사료 등을 판매,
1,접객시설을 갖추고 김밥등 분식 제공,
2,개인택시로 일반인을 대상으로 승객운송서비스,
3,현장에서 고객의 의뢰에 의해 기와로 지붕공사,
4,가정용 철물및 용품을 수리 시설을 갖추고 가정용 보일러 및 하수도를 수리,
...,...,...
9995,펜션에서 일반인을 대상으로 숙박서비스제공,
9996,음식점에서 접객시설을 갖추고 추어탕 판매,
9997,부동산중개소 계약에의거 부동산중개 부동산중개 서비스,
9998,"원단,실 재단,재봉하여 남성복",


In [50]:
train_digit1['target'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])

In [53]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

test_label = df_test_digit1['target'].map(target_to_num_digit1)

tokenized_test = tokenizer(
    list(df_test_digit1['text']),
    return_tensors="pt",
    max_length=64,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

for fold in range(5):

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 225
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/220406_inference_digit1/model' + str(fold), num_labels=19)
  model.resize_token_embeddings(tokenizer.vocab_size)
  accelerator = Accelerator()
  model = accelerator.unwrap_model(model)

  output_pred = []
  output_prob = []

  model, dataloader= accelerator.prepare(model, dataloader)

  model.eval()

  for i, data in enumerate(tqdm(dataloader)):
      with torch.no_grad():
          outputs = model(
              input_ids=data['input_ids'],
              attention_mask=data['attention_mask']
          )
      logits = outputs[0]
      prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      result = np.argmax(logits, axis=-1)
      output_pred.append(result)
      output_prob.append(prob)
      
  pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

  def num_to_label(label):
      label_dict = {v: k for k, v in my_dict1.items()}
      str_label = []

      for i, v in enumerate(label):
          str_label.append([i,label_dict[v]])
      
      return str_label

  answer = num_to_label(pred_answer)

  df_label1 = pd.DataFrame(answer, columns=['index', 'label'])
  df_prob1 = pd.DataFrame(output_prob)

  df_label1.to_csv('/content/drive/MyDrive/220406_inference_digit1/pred_label'+str(fold)+'.csv', index=False)
  df_prob1.to_csv('/content/drive/MyDrive/220406_inference_digit1/pred_prob'+str(fold)+'.csv', index=False)

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

In [54]:
df_label1

,index,label
0,0,G
1,1,I
2,2,H
3,3,F
4,4,S
...,...,...
9995,9995,I
9996,9996,I
9997,9997,L
9998,9998,C


### digit_2

In [56]:
df_test_digit2 = df_test
df_test_digit2

,text,target
0,소매업자를 대상으로 사업장에서 반려동물 사료 등을 판매,
1,접객시설을 갖추고 김밥등 분식 제공,
2,개인택시로 일반인을 대상으로 승객운송서비스,
3,현장에서 고객의 의뢰에 의해 기와로 지붕공사,
4,가정용 철물및 용품을 수리 시설을 갖추고 가정용 보일러 및 하수도를 수리,
...,...,...
9995,펜션에서 일반인을 대상으로 숙박서비스제공,
9996,음식점에서 접객시설을 갖추고 추어탕 판매,
9997,부동산중개소 계약에의거 부동산중개 부동산중개 서비스,
9998,"원단,실 재단,재봉하여 남성복",


In [57]:
df_test_digit2['target'] = ''

In [58]:
train_digit2['target'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71])

In [60]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

test_label = df_test_digit2['target'].map(target_to_num_digit2)

tokenized_test = tokenizer(
    list(df_test_digit2['text']),
    return_tensors="pt",
    max_length=64,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

for fold in range(5):

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 225
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/220406_inference_digit2/model' + str(fold), num_labels=72)
  model.resize_token_embeddings(tokenizer.vocab_size)
  accelerator = Accelerator()
  model = accelerator.unwrap_model(model)

  output_pred = []
  output_prob = []

  model, dataloader= accelerator.prepare(model, dataloader)

  model.eval()

  for i, data in enumerate(tqdm(dataloader)):
      with torch.no_grad():
          outputs = model(
              input_ids=data['input_ids'],
              attention_mask=data['attention_mask']
          )
      logits = outputs[0]
      prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      result = np.argmax(logits, axis=-1)
      output_pred.append(result)
      output_prob.append(prob)
      
  pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

  def num_to_label(label):
      label_dict = {v: k for k, v in my_dict2.items()}
      str_label = []

      for i, v in enumerate(label):
          str_label.append([i,label_dict[v]])
      
      return str_label

  answer = num_to_label(pred_answer)

  df_label2 = pd.DataFrame(answer, columns=['index', 'label'])
  df_prob2 = pd.DataFrame(output_prob)

  df_label2.to_csv('/content/drive/MyDrive/220406_inference_digit2/pred_label'+str(fold)+'.csv', index=False)
  df_prob2.to_csv('/content/drive/MyDrive/220406_inference_digit2/pred_prob'+str(fold)+'.csv', index=False)

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

In [61]:
df_label2

,index,label
0,0,47
1,1,58
2,2,42
3,3,68
4,4,96
...,...,...
9995,9995,58
9996,9996,58
9997,9997,91
9998,9998,49


### digit3 추론

In [62]:
df_test_digit3 = df_test
df_test_digit3

,text,target
0,소매업자를 대상으로 사업장에서 반려동물 사료 등을 판매,
1,접객시설을 갖추고 김밥등 분식 제공,
2,개인택시로 일반인을 대상으로 승객운송서비스,
3,현장에서 고객의 의뢰에 의해 기와로 지붕공사,
4,가정용 철물및 용품을 수리 시설을 갖추고 가정용 보일러 및 하수도를 수리,
...,...,...
9995,펜션에서 일반인을 대상으로 숙박서비스제공,
9996,음식점에서 접객시설을 갖추고 추어탕 판매,
9997,부동산중개소 계약에의거 부동산중개 부동산중개 서비스,
9998,"원단,실 재단,재봉하여 남성복",


In [63]:
df_test_digit3['target'] = ''

In [64]:
train_digit3['target'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [67]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

test_label = df_test_digit3['target'].map(target_to_num_digit3)

tokenized_test = tokenizer(
    list(df_test_digit3['text']),
    return_tensors="pt",
    max_length=64,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

for fold in range(5):

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 225
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/220406_inference_digit3/model' + str(fold), num_labels=219)
  model.resize_token_embeddings(tokenizer.vocab_size)
  accelerator = Accelerator()
  model = accelerator.unwrap_model(model)

  output_pred = []
  output_prob = []

  model, dataloader= accelerator.prepare(model, dataloader)

  model.eval()

  for i, data in enumerate(tqdm(dataloader)):
      with torch.no_grad():
          outputs = model(
              input_ids=data['input_ids'],
              attention_mask=data['attention_mask']
          )
      logits = outputs[0]
      prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      result = np.argmax(logits, axis=-1)
      output_pred.append(result)
      output_prob.append(prob)
      
  pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

  def num_to_label(label):
      label_dict = {v: k for k, v in my_dict3.items()}
      str_label = []

      for i, v in enumerate(label):
          str_label.append([i,label_dict[v]])
      
      return str_label

  answer = num_to_label(pred_answer)

  df_label3 = pd.DataFrame(answer, columns=['index', 'label'])
  df_prob3 = pd.DataFrame(output_prob)

  df_label3.to_csv('/content/drive/MyDrive/220406_inference_digit3/pred_label'+str(fold)+'.csv', index=False)
  df_prob3.to_csv('/content/drive/MyDrive/220406_inference_digit3/pred_prob'+str(fold)+'.csv', index=False)

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

In [70]:
df_label3

,index,label
0,0,471
1,1,463
2,2,562
3,3,478
4,4,473
...,...,...
9995,9995,463
9996,9996,463
9997,9997,721
9998,9998,912


# 가중치 계산

In [88]:
df__test = df_test_1

In [89]:
df__test

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text
90000,id_0237676,G,46,462,소매업자를 대상으로,사업장에서,반려동물 사료 등을 판매,소매업자를 대상으로 사업장에서 반려동물 사료 등을 판매
90001,id_0786773,I,56,561,,접객시설을 갖추고 김밥등 분식 제공,,접객시설을 갖추고 김밥등 분식 제공
90002,id_0908998,H,49,492,개인택시로,일반인을 대상으로,승객운송서비스,개인택시로 일반인을 대상으로 승객운송서비스
90003,id_0694988,F,42,421,현장에서,고객의 의뢰에 의해,기와로 지붕공사,현장에서 고객의 의뢰에 의해 기와로 지붕공사
90004,id_0769876,S,95,953,가정용 철물및 용품을,수리 시설을 갖추고,가정용 보일러 및 하수도를 수리,가정용 철물및 용품을 수리 시설을 갖추고 가정용 보일러 및 하수도를 수리
...,...,...,...,...,...,...,...,...
99995,id_0459934,I,55,551,펜션에서,일반인을 대상으로,숙박서비스제공,펜션에서 일반인을 대상으로 숙박서비스제공
99996,id_0470459,I,56,561,음식점에서,접객시설을 갖추고,추어탕 판매,음식점에서 접객시설을 갖추고 추어탕 판매
99997,id_0832557,L,68,682,부동산중개소,계약에의거 부동산중개,부동산중개 서비스,부동산중개소 계약에의거 부동산중개 부동산중개 서비스
99998,id_0781401,C,14,141,"원단,실","재단,재봉하여",남성복,"원단,실 재단,재봉하여 남성복"


In [90]:
df__test = df__test.reset_index(drop=True)

In [92]:
df__test = df__test[['digit_1','digit_2','digit_3']]
df__test = df__test.reset_index(drop=True)

df__test

,digit_1,digit_2,digit_3
0,G,46,462
1,I,56,561
2,H,49,492
3,F,42,421
4,S,95,953
...,...,...,...
9995,I,55,551
9996,I,56,561
9997,L,68,682
9998,C,14,141


In [93]:
df__test['test_1'] = df_label1['label']
df__test['test_2'] = df_label2['label']
df__test['test_3'] = df_label3['label']

df__test

,digit_1,digit_2,digit_3,test_1,test_2,test_3
0,G,46,462,G,47,471
1,I,56,561,I,58,463
2,H,49,492,H,42,562
3,F,42,421,F,68,478
4,S,95,953,S,96,473
...,...,...,...,...,...,...
9995,I,55,551,I,58,463
9996,I,56,561,I,58,463
9997,L,68,682,L,91,721
9998,C,14,141,C,49,912


In [94]:
len(df__test[df__test['digit_1']==df__test['test_1']])/len(df__test)

0.9668

In [95]:
len(df__test[df__test['digit_2']==df__test['test_2']])/len(df__test)

0.0

In [96]:
len(df__test[df__test['digit_3']==df__test['test_3']])/len(df__test)

0.0

In [97]:
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pandoc is already the newest version (1.19.2.4~dfsg-1build4).
pandoc set to manually installed.
The following additional packages will be installed:
  fonts-droid-fallback fonts-lato fonts-lmodern fonts-noto-mono fonts-texgyre
  javascript-common libcupsfilters1 libcupsimage2 libgs9 libgs9-common
  libijs-0.35 libjbig2dec0 libjs-jquery libkpathsea6 libpotrace0 libptexenc1
  libruby2.5 libsynctex1 libtexlua52 libtexluajit2 libzzip-0-13 lmodern
  poppler-data preview-latex-style rake ruby ruby-did-you-mean ruby-minitest
  ruby-net-telnet ruby-power-assert ruby-test-unit ruby2.5
  rubygems-integration t1utils tex-common tex-gyre texlive-base
  texlive-binaries texlive-fonts-recommended texlive-latex-base
  texlive-latex-recommended texlive-pictures texlive-plain-generic tipa
Suggested packages:
  fonts-noto apache2 | lighttpd | httpd poppler-utils ghostscript
  fonts-japanese-mincho | fonts-ipa

In [100]:
!jupyter nbconvert --to PDF '/content/drive/My Drive/20220406_hj.ipynb'

[NbConvertApp] Converting notebook /content/drive/My Drive/20220406_hj.ipynb to PDF
[NbConvertApp] Writing 419372 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', './notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', './notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 236137 bytes to /content/drive/My Drive/20220406_hj.pdf


In [99]:
1-((len(df)-len(df[(df['label0'] == df['label1']) & (df['label1'] == df['label2']) & (df['label2'] == df['label3']) & (df['label3'] == df['label4'])]))/len(df))

KeyError: ignored

In [ ]:
hard = df[df.merge(df[(df['label0'] == df['label1']) & (df['label1'] == df['label2']) & (df['label2'] == df['label3']) & (df['label3'] == df['label4'])], how = 'left', left_index=True, right_index=True)['text_y'].isnull()]
hard

In [ ]:
pred0 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_prob0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_prob1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_prob2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_prob3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_prob4.csv')

In [ ]:
pred = pd.DataFrame((np.array(pred0) + np.array(pred1) + np.array(pred2) + np.array(pred3) + np.array(pred4))/5)
# test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')
test = pd.concat([test, pred], axis=1)
test.head(3)

In [ ]:
def num_to_label(label):
    label_dict = label_dict = {v: k for k, v in my_dict.items()}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label(np.argmax(np.array(pred), axis=-1))

test['soft'] = pd.DataFrame(answer)[1]
test.head(3)

In [ ]:
soft = test[['label0', 'label1', 'label2', 'label3', 'label4', 'soft']]
soft

In [ ]:
def label_count(df):
  num_count = {}
  cols = ['label0', 'label1', 'label2', 'label3', 'label4']

  for col in cols:
    try: num_count[df[col]] += 1
    except: num_count[df[col]] = 1
  return(num_count)

temp = soft.apply(label_count, axis=1)
soft['temp'] = temp

def label_count(temp:dict):
  keys = []
  temptemp = 0
  for k, v in temp.items():
    if v >= 3:
      temptemp = k
  keys.append(temptemp)
  return keys[0]
  
soft['hard'] = soft['temp'].map(label_count)

In [ ]:
soft[soft['soft'] != soft['hard']]

In [ ]:
df = soft[['label0',	'label1',	'label2',	'label3',	'label4',	'soft',	'hard']]
df

In [ ]:
submission

In [ ]:
print(my_variable)

In [ ]:
def split_label(text:str):
  return text.split(' ')[0]

submission['digit_1'] = df['soft'].map(split_label)

def split_label(text:str):
  return text.split(' ')[1]

submission['digit_2'] = df['soft'].map(split_label)

def split_label(text:str):
  return text.split(' ')[2]

submission['digit_3'] = df['soft'].map(split_label)

In [ ]:
submission

In [ ]:
submission.to_csv('/content/drive/MyDrive/220331_inference/submission.csv', index=False)